In [71]:
import time 
import shadow_useragent
import copy, sys, os, re, random
import glob
import threading
import bs4 as bs
import pandas as pd
import requests
from datetime import date
from fake_useragent import UserAgent
from urllib2 import urlopen, Request
import urllib
import urllib2
from urllib2 import HTTPError
from fake_useragent import UserAgent
from urllib import FancyURLopener
import warnings
warnings.filterwarnings("ignore")

In [72]:
PROXIE_LIST = ['http://144.202.65.136:1440', 'http://144.202.65.136:1441', 'http://144.202.65.136:1442',
              'http://144.202.65.136:1443','http://144.202.65.136:1444']
new_cols = ['Category 1', 'Category 2', 'Category 3', 'Enrollment', 'Language', 'Description', 'Last Update', 'Date Scraped']

In [73]:
def build_driver(user_agent):
    options = webdriver.ChromeOptions()
    PROXY = "144.202.65.136:1440"
    
    options.add_argument("--incognito")
#     options.add_argument("--headless")
    prefs = {'profile.managed_default_content_settings.images':2, 'disk-cache-size': 4096}
    options.add_experimental_option("prefs", prefs)
    options.add_argument('--proxy-server=%s' % PROXY)
    options.binary_location = '/Applications/Brave Browser.app/Contents/MacOS/Brave Browser'
    driver_path = "/Users/thai/Desktop/Kris/chromedriver"
    options.add_argument('user-agent='+user_agent)

    driver = webdriver.Chrome(executable_path=driver_path, chrome_options=options)

    return driver

In [74]:
def scrape_html(html, date_scraped):
    soup = bs.BeautifulSoup(html, 'lxml')
    enrollment = soup.find('div', attrs = {'data-purpose' : 'enrollment'})
    last_update = soup.find('div', attrs = {'class': 'last-update-date'})
    language = soup.find('div', attrs = {'class': 'clp-lead__locale'})
    description = soup.find('div', attrs = {'class' : 'clp-lead__headline'})
    categories = [cat.text.strip("\n") for cat in soup.find_all('a', attrs = {'class': 'topic-menu__link'})]
    
    try:
        data = [categories[0], categories[1], categories[2], enrollment.text.strip(" \n").split(" ")[0], 
                language.text.strip("\n"), description.text.strip("\n"),
                last_update.text.strip("\n").split(" ")[-1], date_scraped]
        
        return data
    except IndexError:
        raise IndexError()

In [75]:
# ua = shadow_useragent.ShadowUserAgent()
# agent = ua.percent(0.05)
# proxy = urllib2.ProxyHandler({'http': random.choice(PROXIE_LIST)})
# opener = urllib2.build_opener(proxy)
# opener.addheaders = [('User-Agent', agent)]
# urllib2.install_opener(opener)

In [76]:
proxy = urllib2.ProxyHandler({'http': random.choice(PROXIE_LIST)})
opener = urllib2.build_opener(proxy)
urllib2.install_opener(opener)

In [77]:
def scrape_single_url(url, proxies, user_agent, date_scraped):
    try:
        headers = {'user-agent': user_agent}
        req = urllib2.Request(url, headers=headers)
        request = urllib2.urlopen(req)
#         headers = {'user-agent': user_agent}
#         req = requests.get(url, headers=headers,proxies=proxies)
#         html = request.read()
        data = None
        return data
    except Exception as e:
        print e
        time.sleep(2)
#         proxies = { 'http' : random.choice(PROXIE_LIST) }
        return scrape_single_url(url, proxies, user_agent, date_scraped)

In [78]:
def scape_v2(url, proxies, user_agent, date_scraped):
    try:
#         request = urllib2.urlopen(url)
        headers = {'user-agent': user_agent}
#                     'Connection': 'close'}
        req = requests.get(url, headers=headers,proxies=proxies)
        req.encoding = 'ISO-8859-1'
        html = req.text
        data = None
        return data
    except Exception as e:
        print e
        time.sleep(2)
#         proxies = { 'http' : random.choice(PROXIE_LIST) }
        return scrape_single_url(url, proxies, user_agent, date_scraped)

In [79]:
def pull_info(csv_in, proxies, user_agent, date_scraped):
    out_file = os.path.basename(csv_in)
    df = pd.read_csv(csv_in)
    if 'Category 1' in df:
        df = df.drop(['Category 1','Category 2', 'Category 3'], axis=1)

    urls = list(df['URL'])
        
    for idx in range(5):
        url = 'http://' + urls[idx]
        print str(idx) + ": " + url
        data = scrape_single_url(url, proxies, user_agent, date_scraped)
#         df.loc[idx,new_cols] = data
#     df.to_csv('~/Desktop/Kris/final/'+out_file, index = None, header=True,encoding = 'utf-8')
    
    print csv_in + " :DONE"

    return df

In [80]:
def pull_info2(csv_in, proxies, user_agent, date_scraped):
    out_file = os.path.basename(csv_in)
    df = pd.read_csv(csv_in)
    for i in new_cols:
        df[i] = pd.Series()

    urls = list(df['URL'])
        
    for idx in range(5):
        url = 'http://' + urls[idx]
        print str(idx) + ": " + url
        data = scape_v2(url, proxies, user_agent, date_scraped)
#         df.loc[idx,new_cols] = data
#     df.to_csv('~/Desktop/Kris/final/'+out_file, index = None, header=True,encoding = 'utf-8')
    
    print csv_in + " :DONE"

    return df

In [81]:
def task(csv):
    ua = shadow_useragent.ShadowUserAgent()
    agent = ua.percent(0.05)
    proxies = {'http': random.choice(PROXIE_LIST)}
    today = date.today()
    date_string = today.strftime("%m/%d/%y")
    df = pull_info(csv, proxies, agent, date_string)

In [82]:
def task2(csv):
    ua = shadow_useragent.ShadowUserAgent()
    agent = ua.percent(0.05)
    proxies = {'http': random.choice(PROXIE_LIST)}
    today = date.today()
    date_string = today.strftime("%m/%d/%y")
    df = pull_info2(csv, proxies, agent, date_string)

In [ ]:
# proxies = { 'http' : random.choice(PROXIE_LIST) }
f = '/Users/thai/Desktop/Kris/out/Procreate Digital Illustration App.csv'
t0 = time.time()
task(f)
t1 = time.time()
print t1 - t0

In [ ]:
glob.glob('/Users/thai/Desktop/Kris/out'+ "/*.csv")[83]

In [85]:
from multiprocessing import Pool

In [86]:
f = ['/Users/thai/Desktop/Kris/out/Procreate Digital Illustration App.csv','/Users/thai/Desktop/Kris/out/Microsoft Access.csv',
     '/Users/thai/Desktop/Kris/out/CompTIA A+.csv']
t0 = time.time()
p = Pool(10)  # Pool tells how many at a time
records = p.map(task2, f)
p.terminate()
p.join()
t1 = time.time()
print t1 - t0

0: http://www.udemy.com/course/drawing-and-painting-on-the-ipad-with-procreate/
0: http://www.udemy.com/course/new-comptia-a-2019-certification-1001-the-total-course/
0: http://www.udemy.com/course/microsoftaccess/
1: http://www.udemy.com/course/microsoft-access-complete-beginner-to-advanced/
1: http://www.udemy.com/course/procreate-draw-sketch-paint-and-design-on-your-ipad/
1: http://www.udemy.com/course/new-comptia-a-2019-certification-1002-the-total-course/
2: http://www.udemy.com/course/import-excel-into-access/
2: http://www.udemy.com/course/procreate-guide/
2: http://www.udemy.com/course/comptia-a-certification-1001-practice-test/
3: http://www.udemy.com/course/procreate-course-how-to-draw-on-your-ipad-how-to-paint-on-your-ipad/
3: http://www.udemy.com/course/microsoft-access-2016-access-from-beginner-to-advanced/
3: http://www.udemy.com/course/comptia-a-certification-1002-practice-tests/
4: http://www.udemy.com/course/how-to-draw-cartoon-characters-in-procreate/
4: http://www.ud

In [83]:
f = '/Users/thai/Desktop/Kris/out/Procreate Digital Illustration App.csv'
t0 = time.time()
task2(f)
t1 = time.time()
print t1 - t0

0: http://www.udemy.com/course/drawing-and-painting-on-the-ipad-with-procreate/
1: http://www.udemy.com/course/procreate-draw-sketch-paint-and-design-on-your-ipad/
2: http://www.udemy.com/course/procreate-guide/
3: http://www.udemy.com/course/procreate-course-how-to-draw-on-your-ipad-how-to-paint-on-your-ipad/
4: http://www.udemy.com/course/how-to-draw-cartoon-characters-in-procreate/
/Users/thai/Desktop/Kris/out/Procreate Digital Illustration App.csv :DONE
9.03328514099


In [ ]:
THREADS = 1

In [ ]:
glob.glob('/Users/thai/Desktop/Kris/out'+ "/*.csv")[90]

In [ ]:
# done = [os.path.split(s)[1] for s in glob.glob(os.getcwd() + "/final/*.csv")]
# all_files = glob.glob('/Users/thai/Desktop/Kris/out'+ "/*.csv")
# counter = 0
# thread_pool = []
# proxies = { 'http' : random.choice(PROXIE_LIST) }
# f = '/Users/thai/Desktop/Kris/out/Procreate Digital Illustration App.csv'
# task(f, proxies)
# for f in all_files:
#     file_name = os.path.split(f)[1]
#     if file_name not in done:
#         print "Starting " + file_name
#         proxies = { 'http' : random.choice(PROXIE_LIST) }
#         if counter == THREADS:
#             break
#         counter += 1
#         task(f, proxies)
#         thread_pool.append(threading.Thread(target=task, args=(f,proxies, )))

In [ ]:
ua = shadow_useragent.ShadowUserAgent()
agent = ua.percent(0.05)

In [ ]:
driver = build_driver(agent)
f = '/Users/thai/Desktop/Kris/out/Procreate Digital Illustration App.csv'
driver.get('https://' + f)
time.sleep(2)
html = driver.page_source
print html
# driver.quit()
# today = date.today()
# date_string = today.strftime("%m/%d/%y")
# data = scrape_html(html, date_string)

In [ ]:
# for thread in thread_pool:
#     time.sleep(1)
#     thread.start()

In [ ]:
# for thread in thread_pool:
#     thread.join()

In [62]:
data = ['col1', 'col2', 'col3', 'enroll','lang','des','date1','date2']

In [63]:
data_dict = dict(zip(new_cols, data))

In [64]:
data_dict

{'Category 1': 'col1',
 'Category 2': 'col2',
 'Category 3': 'col3',
 'Date Scraped': 'date2',
 'Description': 'des',
 'Enrollment': 'enroll',
 'Language': 'lang',
 'Last Update': 'date1'}

In [65]:
df_test = pd.read_csv('/Users/thai/Desktop/Kris/out/Procreate Digital Illustration App.csv')

In [66]:
if 'Category 1' in df_test:
    df_test = df_test.drop(['Category 1','Category 2', 'Category 3'], axis=1)

In [67]:
new_frame = pd.DataFrame(columns=new_cols)

In [68]:
for i in df_test.index:
    new_frame = new_frame.append(data_dict, ignore_index=True)

In [69]:
df_test = pd.concat([df_test, new_frame], axis=1)

In [70]:
df_test

,Course,URL,instructor,Price (full),Price (discount),Rank,Category 1,Category 2,Category 3,Enrollment,Language,Description,Last Update,Date Scraped
0,Drawing and Painting on the iPad with Procreate,www.udemy.com/course/drawing-and-painting-on-t...,Brad Colbow,19.99,11.99,"4.6(6,663 ratings)",col1,col2,col3,enroll,lang,des,date1,date2
1,"Procreate: Draw, Sketch, Paint, and Design on ...",www.udemy.com/course/procreate-draw-sketch-pai...,Video School Online Inc,199.99,16.79,"4.5(1,058 ratings)",col1,col2,col3,enroll,lang,des,date1,date2
2,The Beginner's Guide to Digital Art with Procr...,www.udemy.com/course/procreate-guide/,Kurt Michael Russell,49.99,12.59,"4.6(1,322 ratings)",col1,col2,col3,enroll,lang,des,date1,date2
3,ProCreate Masterclass: How to Draw and Paint o...,www.udemy.com/course/procreate-course-how-to-d...,Scott Harris,199.99,11.99,"4.5(1,001 ratings)",col1,col2,col3,enroll,lang,des,date1,date2
4,How to Draw Cartoon Characters In Procreate,www.udemy.com/course/how-to-draw-cartoon-chara...,Brad Colbow,19.99,11.39,4.7(573 ratings),col1,col2,col3,enroll,lang,des,date1,date2
5,The Procreate 4.1 Super Course,www.udemy.com/course/the-procreate-4-super-cou...,Austin Batchelor,119.99,11.39,4.5(621 ratings),col1,col2,col3,enroll,lang,des,date1,date2
6,The Digital Painting w/ Procreate Guide | Gray...,www.udemy.com/course/procreatepainting/,Kurt Michael Russell,49.99,11.39,4.7(396 ratings),col1,col2,col3,enroll,lang,des,date1,date2
7,"Procreate Academy. Ultimate Course, Digital Dr...",www.udemy.com/course/procreate-academy_procrea...,Andres Brachetta ⭐⭐⭐⭐⭐,199.99,14.39,4.2(90 ratings),col1,col2,col3,enroll,lang,des,date1,date2
8,The Advanced Guide to Digital Art with Procrea...,www.udemy.com/course/procreate-guide-advanced/,Kurt Michael Russell,49.99,11.39,4.5(246 ratings),col1,col2,col3,enroll,lang,des,date1,date2
9,Procreate like a Pro: Create Awesome Digital A...,www.udemy.com/course/procreate-like-a-pro-crea...,Robert Marzullo,199.99,13.79,4.7(156 ratings),col1,col2,col3,enroll,lang,des,date1,date2
